In [ ]:
# %% Deep learning - Section 10.91
#    Activation functions comparison

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Load and prepare data

# Load
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url,sep=';')

# Remove some outliers (see lec. 82 for why)
data = data[data['total sulfur dioxide']<200]

# Z-score all the variables but quality
cols2zscore = data.keys()
cols2zscore = cols2zscore.drop('quality')

for col in cols2zscore:
    mean_val  = np.mean(data[col])
    std_val   = np.std(data[col])
    data[col] = (data[col] - mean_val) / std_val

# Binarise quality
data.loc[:,'boolean_quality'] = 0
data.loc[data['quality']>5, 'boolean_quality'] = 1
data.loc[data['quality']<6, 'boolean_quality'] = 0 # Implicit but here for clarity

# Convert from pandas dataframe to PyTorch tensor
data_t = torch.tensor( data[cols2zscore].values ).float()
labels = torch.tensor( data['boolean_quality'].values ).float()

print(f'Data shape: {data_t.shape}')
print(f'Labels shape: {labels.shape}')

# Labels need to be multidimentional for PyTorch, not an array, and need to be long integers too
labels = labels[:,None]
print(f'Proper labels shape: {labels.shape}')


In [ ]:
# %% Split into train and test data

# Split with scikitlearn
train_data,test_data,train_labels,test_labels = train_test_split(data_t,labels,test_size=0.1)

# Convert into PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Model class

# Optional arg to switch activation function
class ANN_wine(nn.Module):
    def __init__(self,act_fun):
        super().__init__()

        # Layers
        self.input  = nn.Linear(11,32)
        self.hid1   = nn.Linear(32,32)
        self.hid2   = nn.Linear(32,32)
        self.output = nn.Linear(32,1)

        # Activation function
        self.act_fun = act_fun

    # Forward propagation
    def forward(self,x):

        act_fun = getattr(torch.nn.functional,self.act_fun)
        x = act_fun(self.input(x))
        x = act_fun(self.hid1(x))
        x = act_fun(self.hid2(x))
        x = self.output(x)

        return x


In [ ]:
# %% Test model

# Use for ex. sigmoid, relu, or tanh
ANN = ANN_wine('sigmoid')
ANN( torch.randn(10,11) ).shape


In [ ]:
# %% Function to train the model

# Parameters
num_epochs = 1000

def train_model():

    # Loss function and optimizer
    loss_fun  = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    # Initialise losses
    train_acc = []
    test_acc  = []
    losses    = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Switch training mode on
        ANN.train()

        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss (with batchnorm arg)
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Only now do backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Batch training accuracy
            batch_acc.append( 100*torch.mean(((yHat>0) == y).float()).item() )
            batch_loss.append(loss.item())

        # Average accuracy from batch
        train_acc.append(np.mean(batch_acc))
        losses.append(np.mean(batch_loss))

        # Test accuracy
        ANN.eval()
        X,y = next(iter(test_loader))
        with torch.no_grad():
            yHat = ANN(X)
        test_acc.append( 100*torch.mean(((yHat>0) == y).float()).item() )

    # Function output
    return train_acc,test_acc,losses


In [ ]:
# %% Parametric experiment on activation functions

activation_funs = [ 'relu','leaky_relu','tanh','sigmoid', ]

train_funs = np.zeros((num_epochs,len(activation_funs)))
test_funs  = np.zeros((num_epochs,len(activation_funs)))

for a,fun in enumerate(activation_funs):

    ANN = ANN_wine(fun)
    train_acc,test_acc,losses = train_model()

    train_funs[:,a] = train_acc
    test_funs[:,a]  = test_acc


In [ ]:
# %% Functions for 1D smoothing filter

# Improved for edge effects - adaptive window
def smooth_adaptive(x,k):
    smoothed = np.zeros_like(x)
    half_k   = k // 2

    for i in range(len(x)):
        start       = max(0, i-half_k)
        end         = min(len(x), i+half_k + 1)
        smoothed[i] = np.mean(x[start:end])

    return smoothed


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(activation_funs)))
for i in range(len(activation_funs)):
    ax[0].plot(smooth_adaptive(train_funs[:,i],20),color=cmaps[i])
    ax[1].plot(smooth_adaptive(test_funs[:,i],20),color=cmaps[i])

ax[0].set_title('Train accuracy')
ax[1].set_title('Test accuracy')

# Make the legend easier to read
leglabels = [fun for i,fun in enumerate(activation_funs)]

# Common features
for i in range(2):
    ax[i].legend(leglabels)
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Accuracy (%)')
    ax[i].set_ylim([30,101])
    ax[i].grid()

plt.savefig('figure43_activation_functions_comparison.png')

plt.show()

files.download('figure43_activation_functions_comparison.png')


In [ ]:
# %% Exercise 1
#    Lower the learning rate to .001 (instead of .01), and increase the number of epochs to 1500.
#    Do the conclusions about the activation functions change?

# In this case the sigmoid function is no longer able to perform above chance
# level, not even with more epochs. Other activation functions underperform a bit too


In [ ]:
# %% Exercise 2
#    Most of accuracy plots in this course show train and test in separate axes. Modify the visualization code to show
#    both train and test accuracy in the same plot. What additional visualization considerations are there?

# I guess some differences in the plotting line style to make the plot understandable

# Plotting
phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,1,figsize=(6*phi,6))

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(activation_funs)))
for i in range(len(activation_funs)):
    ax.plot(smooth_adaptive(train_funs[:,i],20),color=cmaps[i])
    ax.plot(smooth_adaptive(test_funs[:,i],20),'--',color=cmaps[i])

ax.set_title('Train and test accuracy')

# Make the legend easier to read
leglabels = [fun for fun in activation_funs for _ in range(2)]

# Common features
ax.legend(leglabels)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy (%)')
ax.set_ylim([30,101])
ax.grid()

plt.savefig('figure45_activation_functions_comparison_extra2.png')

plt.show()

files.download('figure45_activation_functions_comparison_extra2.png')
